# DB Queries

In [ ]:
from neo4j import GraphDatabase

URI = "bolt://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "neo4jproject123"

DBS = [
    ("db1", 1),
    ("db2", 2),
    ("db3", 3),
]

### Query a

In [ ]:
query_a = """
MATCH (m:Customer)-[:USE]->(t:Terminal)<-[:USE]-(n:Customer)
WHERE m <> n
WITH m, n, COUNT(DISTINCT t) AS sharedTerminals
WHERE sharedTerminals >= 4

MATCH (m)-[:PERFORM]->(tm:Transaction)
MATCH (n)-[:PERFORM]->(tn:Transaction)
WITH
    m, n, sharedTerminals,
    COUNT(DISTINCT tm) AS m_tx_count,
    COUNT(DISTINCT tn) AS n_tx_count

WHERE abs(m_tx_count - n_tx_count) <= 2

RETURN
    m.id AS customer_M,
    n.id AS customer_N,
    sharedTerminals,
    m_tx_count,
    n_tx_count
ORDER BY sharedTerminals DESC;
"""

### Query b

In [ ]:
query_b = """
MATCH (t:Terminal)<-[:ON]-(tr:Transaction)
WITH t, tr, datetime(tr.dateTime) AS dt
WITH t, tr, dt,
     tr.amount AS amount,
     dt.year AS y,
     ceil(dt.month / 3.0) AS q

MATCH (t)<-[:ON]-(prev:Transaction)
WITH t, tr, dt, amount, y, q,
     datetime(prev.dateTime) AS pdt,
     prev.amount AS p_amount
WHERE pdt.year = y AND ceil(pdt.month / 3.0) = q - 1

WITH t, tr, dt, amount, percentileCont(p_amount, 0.5) AS median_prev
WHERE amount > median_prev * 1.3

RETURN t.id AS terminal,
       tr.id AS transaction,
       amount AS tx_amount,
       median_prev AS prev_quarter_median,
       "potential outlier" AS flag
ORDER BY dt;
"""

### Query c

In [ ]:
query_c = """
MATCH (u:Customer {id: $u_id})-[:PERFORM]->(:Transaction)-[:ON]->(t1:Terminal)
MATCH (t1)<-[:ON]-(:Transaction)<-[:PERFORM]-(u2:Customer)
WHERE u2 <> u

MATCH (u2)-[:PERFORM]->(:Transaction)-[:ON]->(t2:Terminal)
MATCH (t2)<-[:ON]-(:Transaction)<-[:PERFORM]-(u3:Customer)
WHERE u3 <> u AND u3 <> u2

RETURN DISTINCT u.id AS startCustomer,
                u2.id AS level2Customer,
                u3.id AS reachableCustomerDegree3
"""

### Run on DBs

In [ ]:
def run_on_dbs(db_name, query, params=None, show_limit=50):
    """Run `query` on each DB in `DBS`, print up to `show_limit` rows per DB."""
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    try:
        
        print(f"\n--- Running on {db_name} ---")
        with driver.session(database=db_name) as session:
            result = session.run(query, params or {})
            records = list(result)
        
            print(f"Returned {len(records)} records (showing up to {show_limit}):")
    
            for r in records[:show_limit]:
                try:
                    print(dict(r))
                except Exception:
                    print(r)
    
    finally:
        driver.close()


In [ ]:
run_on_dbs("db1", query_a)

In [ ]:
run_on_dbs("db2", query_b)

In [ ]:
run_on_dbs("db3", query_c, params={"u_id": "5"})